In [22]:
# %%file strategy_2.py

import import_ipynb
from strategy_base import TradeStrategyBase

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class TradeStrategy2(TradeStrategyBase):
    """
    Trade Strategy 2: Mean Reversion. 
    When price falls two days in a row, and cumulatively more than x percent (_p_change_threshold) in total, default to 10%.
    Buy and hold for a number of days (_hold_stock_threshold) days, default to 20. 
    """
    # Absolute threshold for buy in
    _p_change_threshold = 10
    # Number of days to hold after Buy
    _hold_stock_threshold = 10
    
    def __init__(self, p_change_threshold:float = 10.0, hold_stock_threshold:int = 10, should_log:bool = False, should_plot:bool = False, axs = None):
        """
        Initialize Strategy with given parameters
        :type p_change_threshold: float
        :type hold_stock_threshold: int 
        :type should_log: bool
        :type should_plot: bool
        :param p_change_threshold: Absolute threshold for a stock to rise in two days. E.g. 7.5% = 7.5 
        :param hold_stock_threshold: Number of days to hold a stock. E.g. 20 days = 20
        :param should_log Whether or not log the strategy
        :param should_plot Whether or not plot the strategy
        """
        self._p_change_threshold = p_change_threshold
        self._hold_stock_threshold = hold_stock_threshold
        self._should_log = should_log
        self._should_plot = should_plot
        self._axs = axs
        
        self._held_tradedays = pd.DataFrame()
        self._previous_tradeday = None
        self._held_period_days = 0
        
    def trade(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Perform Analysis, Decide on Trade, and Execute Trade
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        :return: 
        """
            
        """
        Firstly, Increment holding date count
        """
        if self._held_period_days != 0:
            # Increment date
            self._held_period_days += 1
            # Is currently holding, attach new tradeday in DataFrame
            self._held_tradedays = self._held_tradedays.append(tradeday)
        
        """
        Secondly, Buying
        """
        if self._evaluate_buying(date=date,
                                 tradeday=tradeday):
            self._execute_buying(date=date,
                                 tradeday=tradeday)
        
        """
        Lastly, Selling
        """
        if self._evaluate_selling(date=date,
                                  tradeday=tradeday):
            self._execute_selling(date=date,
                                  tradeday=tradeday)
        
    def _evaluate_buying(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Evaluate whether or not BUY should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        trade_day: pd.Series = tradeday.copy() # Create a copy() in case we need to change it.
        if self._held_period_days == 0 and self._previous_tradeday is not None:
            # trade_day.change < 0 : Tells if price dropped
            is_today_price_down = trade_day.p_change < 0
            # Check if yesterday was down
            is_yesterday_price_down = self._previous_tradeday.p_change  < 0
            
            # Sum of all drops
            total_down_rate = trade_day.p_change + self._previous_tradeday.p_change
        
            # Set previous_tradeday to today
            self._previous_tradeday = tradeday
            
            if is_today_price_down and \
                is_yesterday_price_down and \
                    np.abs(total_down_rate) > self._p_change_threshold:
                return True
            else:
                return False
        else:
            # Set previous_tradeday to today
            self._previous_tradeday = tradeday
            return False
        
    def _execute_buying(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        How BUY should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        if self._should_log:
            print("Bought on {} at ${}.".format(date, tradeday.close))
        self._held_period_days += 1 # Start holding date count
        self._held_tradedays = pd.DataFrame(tradeday).T # Transpose of Series
        self._last_buy_date = date

    def _evaluate_selling(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Evaluate whether or not SELL should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        if self._held_period_days >= self._hold_stock_threshold:
            # When holding period exceeds hold_stock_threshold, sell stocks
            return True
        else:
            return False
            
    def _execute_selling(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        How SELL should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """ 
        if self._should_log:
            print("Sold on {} at ${}.".format(date, tradeday.close))
        if self._should_plot:
            self._plot_trade(axs=self._axs,
                             buy_date=self._last_buy_date, 
                             sell_date=date)
        self._previous_tradeday = None # Reset last tradeday
        self._held_period_days = 0  # Reset holding count 
        self._last_buy_date = None # Reset last buy date

    def _plot_trade(self, axs, buy_date, sell_date):

        print("Todo: Plotting {} {}".format(buy_date, sell_date))

        # buy_tradeday = self._held_tradedays[buy_date.date()]
        # sell_tradeday = self._held_tradedays[sell_date.date()]
        # print("Bought at {}, Sold at {}".format(buy_tradeday.close, sell_tradeday.close))

        # drawer = plt if axs is None else axs
        # 
        # # Mark red for up and green for down, slicing using start and end
        # if sell_tradeday.close < buy_tradeday.close:
        #   drawer.fill_between(self._held_tradedays.index[buy_date:sell_date], 0,
        #                   self._held_tradedays['close'][buy_date:sell_date], color='red',
        #                   alpha=.38)
        #   is_win = False
        # else:
        #   drawer.fill_between(amex_df.index[buy_date:sell_date], 0,
        #                   amex_df['close'][buy_date:sell_date], color='green',
        #                   alpha=.38)
        #   is_win = True
        # 
        # return is_win
        
    @property
    def trade_profit(self):
        if 'p_change' in self._held_tradedays.columns:
            profit_array = self._held_tradedays.p_change/100 + 1
            profit_rate = np.product(profit_array)-1
            return profit_rate
        else:
            return 0
    
    @property
    def p_change_threshold(self):
        return self._p_change_threshold
    
    @property
    def keep_stock_threshold(self):
        return self._hold_stock_threshold
        